## Preprocessing

In [5]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [6]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [7]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [8]:
# Look at APPLICATION_TYPE value counts for binning
app_count = application_df['APPLICATION_TYPE'].value_counts()
app_count

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [9]:
apps_less_than_10 = app_count[app_count<100]

In [10]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = apps_less_than_10.index
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: APPLICATION_TYPE, dtype: int64

In [11]:
# Look at CLASSIFICATION value counts for binning
class_count = application_df['CLASSIFICATION'].value_counts()
class_count

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [12]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_greater_than_1 = class_count[class_count>1]
class_greater_than_1

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [13]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = class_count[class_count<1000].index

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [14]:
# grab all columns with dtype object
obj_cols = application_df.select_dtypes(include=[object])
obj_cols.columns

Index(['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE',
       'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS'],
      dtype='object')

In [15]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df, columns=obj_cols.columns)

In [16]:
# Split our preprocessed data into our features and target arrays
X = application_df.drop(columns="IS_SUCCESSFUL")
y = application_df["IS_SUCCESSFUL"]

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [17]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [18]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=16, activation="relu", input_dim=44))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=16, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 16)                720       
                                                                 
 dense_4 (Dense)             (None, 16)                272       
                                                                 
 dense_5 (Dense)             (None, 1)                 17        
                                                                 
Total params: 1,009
Trainable params: 1,009
Non-trainable params: 0
_________________________________________________________________


In [19]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [20]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 1s 564us/step - loss: 0.5908 - accuracy: 0.7072
Epoch 2/50
804/804 [==============================] - 0s 564us/step - loss: 0.5600 - accuracy: 0.7262
Epoch 3/50
804/804 [==============================] - 0s 574us/step - loss: 0.5560 - accuracy: 0.7285
Epoch 4/50
804/804 [==============================] - 0s 571us/step - loss: 0.5536 - accuracy: 0.7292
Epoch 5/50
804/804 [==============================] - 0s 556us/step - loss: 0.5526 - accuracy: 0.7288
Epoch 6/50
804/804 [==============================] - 0s 550us/step - loss: 0.5509 - accuracy: 0.7295
Epoch 7/50
804/804 [==============================] - 0s 552us/step - loss: 0.5503 - accuracy: 0.7294
Epoch 8/50
804/804 [==============================] - 0s 555us/step - loss: 0.5496 - accuracy: 0.7297
Epoch 9/50
804/804 [==============================] - 0s 581us/step - loss: 0.5492 - accuracy: 0.7308
Epoch 10/50
804/804 [==============================] - 0s 570us/step - loss: 0.548

In [21]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5567 - accuracy: 0.7287 - 180ms/epoch - 671us/step
Loss: 0.5566970109939575, Accuracy: 0.7287463545799255


In [22]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=16, activation="relu", input_dim=44))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=16, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=16, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 16)                720       
                                                                 
 dense_7 (Dense)             (None, 16)                272       
                                                                 
 dense_8 (Dense)             (None, 16)                272       
                                                                 
 dense_9 (Dense)             (None, 1)                 17        
                                                                 
Total params: 1,281
Trainable params: 1,281
Non-trainable params: 0
_________________________________________________________________


In [23]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [25]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5564 - accuracy: 0.7202 - 186ms/epoch - 694us/step
Loss: 0.5564320087432861, Accuracy: 0.7202332615852356


In [26]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=32, activation="relu", input_dim=44))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=32, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=32, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 32)                1440      
                                                                 
 dense_11 (Dense)            (None, 32)                1056      
                                                                 
 dense_12 (Dense)            (None, 32)                1056      
                                                                 
 dense_13 (Dense)            (None, 1)                 33        
                                                                 
Total params: 3,585
Trainable params: 3,585
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
804/804 [==============================] - 1s 700us/step - loss: 0.5733 - accuracy: 0.7184
Epoch 2/50
804/804 [==============================] - 1s 748us/step - loss: 0.5558

In [27]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5569 - accuracy: 0.7270 - 195ms/epoch - 729us/step
Loss: 0.5569499731063843, Accuracy: 0.7269970774650574


In [47]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=20, activation="relu", input_dim=44))

for i in range(3):
    nn.add(tf.keras.layers.Dense(units=20, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50, batch_size = 32, verbose = 2)

model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_65 (Dense)            (None, 300)               13500     
                                                                 
 dense_66 (Dense)            (None, 300)               90300     
                                                                 
 dense_67 (Dense)            (None, 300)               90300     
                                                                 
 dense_68 (Dense)            (None, 300)               90300     
                                                                 
 dense_69 (Dense)            (None, 300)               90300     
                                                                 
 dense_70 (Dense)            (None, 300)               90300     
                                                                 
 dense_71 (Dense)            (None, 300)             

In [30]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh', 'sigmoid'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=20,
        step=5), activation=activation, input_dim=44))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 3)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=20,
            step=5),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [31]:
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=50,
    overwrite = True,
    directory = 'project2',
    hyperband_iterations=2)

In [32]:
# run tuner to determine best model
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

Trial 180 Complete [00h 00m 32s]
val_accuracy: 0.7322449088096619

Best val_accuracy So Far: 0.7343440055847168
Total elapsed time: 00h 21m 37s
INFO:tensorflow:Oracle triggered exit


In [33]:
# output the best parameters
top_params = tuner.get_best_hyperparameters(1)
top_params[0].values

{'activation': 'tanh',
 'first_units': 6,
 'num_layers': 2,
 'units_0': 11,
 'units_1': 11,
 'units_2': 16,
 'tuner/epochs': 17,
 'tuner/initial_epoch': 6,
 'tuner/bracket': 2,
 'tuner/round': 1,
 'tuner/trial_id': '0062'}

In [35]:
# train a new model using the optimum paramters
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=11, activation="tanh", input_dim=44))

# 2 layers per our best parameters
nn.add(tf.keras.layers.Dense(units=11, activation="tanh"))
nn.add(tf.keras.layers.Dense(units=16, activation="tanh"))
# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=400, batch_size = 32, verbose = 2)

model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 11)                495       
                                                                 
 dense_8 (Dense)             (None, 11)                132       
                                                                 
 dense_9 (Dense)             (None, 16)                192       
                                                                 
 dense_10 (Dense)            (None, 1)                 17        
                                                                 
Total params: 836
Trainable params: 836
Non-trainable params: 0
_________________________________________________________________
Epoch 1/400
804/804 - 1s - loss: 0.5860 - accuracy: 0.7121 - 780ms/epoch - 971us/step
Epoch 2/400
804/804 - 0s - loss: 0.5627 - accuracy: 0.7284 - 448ms/epoch - 557us/step
Epoch 3/400
804/

Epoch 85/400
804/804 - 0s - loss: 0.5389 - accuracy: 0.7358 - 397ms/epoch - 494us/step
Epoch 86/400
804/804 - 0s - loss: 0.5390 - accuracy: 0.7378 - 402ms/epoch - 500us/step
Epoch 87/400
804/804 - 0s - loss: 0.5388 - accuracy: 0.7362 - 402ms/epoch - 500us/step
Epoch 88/400
804/804 - 0s - loss: 0.5388 - accuracy: 0.7376 - 400ms/epoch - 497us/step
Epoch 89/400
804/804 - 0s - loss: 0.5387 - accuracy: 0.7381 - 400ms/epoch - 497us/step
Epoch 90/400
804/804 - 0s - loss: 0.5388 - accuracy: 0.7370 - 400ms/epoch - 497us/step
Epoch 91/400
804/804 - 0s - loss: 0.5386 - accuracy: 0.7362 - 405ms/epoch - 504us/step
Epoch 92/400
804/804 - 0s - loss: 0.5386 - accuracy: 0.7364 - 405ms/epoch - 504us/step
Epoch 93/400
804/804 - 0s - loss: 0.5388 - accuracy: 0.7372 - 400ms/epoch - 497us/step
Epoch 94/400
804/804 - 0s - loss: 0.5386 - accuracy: 0.7372 - 403ms/epoch - 501us/step
Epoch 95/400
804/804 - 0s - loss: 0.5388 - accuracy: 0.7374 - 403ms/epoch - 501us/step
Epoch 96/400
804/804 - 0s - loss: 0.5382 - 

Epoch 179/400
804/804 - 0s - loss: 0.5363 - accuracy: 0.7382 - 401ms/epoch - 498us/step
Epoch 180/400
804/804 - 0s - loss: 0.5364 - accuracy: 0.7371 - 398ms/epoch - 495us/step
Epoch 181/400
804/804 - 0s - loss: 0.5361 - accuracy: 0.7395 - 405ms/epoch - 503us/step
Epoch 182/400
804/804 - 0s - loss: 0.5364 - accuracy: 0.7386 - 400ms/epoch - 497us/step
Epoch 183/400
804/804 - 0s - loss: 0.5364 - accuracy: 0.7388 - 403ms/epoch - 501us/step
Epoch 184/400
804/804 - 0s - loss: 0.5363 - accuracy: 0.7371 - 401ms/epoch - 498us/step
Epoch 185/400
804/804 - 0s - loss: 0.5362 - accuracy: 0.7384 - 401ms/epoch - 498us/step
Epoch 186/400
804/804 - 0s - loss: 0.5362 - accuracy: 0.7385 - 399ms/epoch - 496us/step
Epoch 187/400
804/804 - 0s - loss: 0.5360 - accuracy: 0.7384 - 403ms/epoch - 501us/step
Epoch 188/400
804/804 - 0s - loss: 0.5361 - accuracy: 0.7387 - 401ms/epoch - 498us/step
Epoch 189/400
804/804 - 0s - loss: 0.5362 - accuracy: 0.7382 - 400ms/epoch - 498us/step
Epoch 190/400
804/804 - 0s - los

804/804 - 0s - loss: 0.5351 - accuracy: 0.7380 - 399ms/epoch - 496us/step
Epoch 273/400
804/804 - 0s - loss: 0.5352 - accuracy: 0.7388 - 401ms/epoch - 499us/step
Epoch 274/400
804/804 - 0s - loss: 0.5351 - accuracy: 0.7404 - 404ms/epoch - 503us/step
Epoch 275/400
804/804 - 0s - loss: 0.5353 - accuracy: 0.7385 - 398ms/epoch - 495us/step
Epoch 276/400
804/804 - 0s - loss: 0.5351 - accuracy: 0.7395 - 406ms/epoch - 504us/step
Epoch 277/400
804/804 - 0s - loss: 0.5351 - accuracy: 0.7392 - 398ms/epoch - 495us/step
Epoch 278/400
804/804 - 0s - loss: 0.5350 - accuracy: 0.7390 - 402ms/epoch - 499us/step
Epoch 279/400
804/804 - 0s - loss: 0.5349 - accuracy: 0.7394 - 396ms/epoch - 493us/step
Epoch 280/400
804/804 - 0s - loss: 0.5351 - accuracy: 0.7394 - 401ms/epoch - 499us/step
Epoch 281/400
804/804 - 0s - loss: 0.5351 - accuracy: 0.7392 - 406ms/epoch - 505us/step
Epoch 282/400
804/804 - 0s - loss: 0.5351 - accuracy: 0.7381 - 400ms/epoch - 498us/step
Epoch 283/400
804/804 - 0s - loss: 0.5350 - ac

Epoch 366/400
804/804 - 0s - loss: 0.5344 - accuracy: 0.7396 - 399ms/epoch - 497us/step
Epoch 367/400
804/804 - 0s - loss: 0.5345 - accuracy: 0.7399 - 394ms/epoch - 490us/step
Epoch 368/400
804/804 - 0s - loss: 0.5345 - accuracy: 0.7395 - 395ms/epoch - 492us/step
Epoch 369/400
804/804 - 0s - loss: 0.5344 - accuracy: 0.7395 - 397ms/epoch - 494us/step
Epoch 370/400
804/804 - 0s - loss: 0.5345 - accuracy: 0.7385 - 398ms/epoch - 494us/step
Epoch 371/400
804/804 - 0s - loss: 0.5342 - accuracy: 0.7391 - 400ms/epoch - 497us/step
Epoch 372/400
804/804 - 0s - loss: 0.5343 - accuracy: 0.7388 - 398ms/epoch - 495us/step
Epoch 373/400
804/804 - 0s - loss: 0.5344 - accuracy: 0.7402 - 400ms/epoch - 497us/step
Epoch 374/400
804/804 - 0s - loss: 0.5344 - accuracy: 0.7401 - 397ms/epoch - 494us/step
Epoch 375/400
804/804 - 0s - loss: 0.5342 - accuracy: 0.7390 - 395ms/epoch - 492us/step
Epoch 376/400
804/804 - 0s - loss: 0.5344 - accuracy: 0.7397 - 393ms/epoch - 489us/step
Epoch 377/400
804/804 - 0s - los

In [36]:
# Export our model to HDF5 file
nn.save("yelp_model.h5")